In [1]:
import os, csv, pandas as pd
from py2neo import Graph

In [2]:
files_path = "../Output/DataLLM"
files = os.listdir(files_path)

In [3]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "@Martiale01"))

In [4]:
for data in files :
    df = pd.read_csv(f"{files_path}/{data}", delimiter=",")
    for _, row in df.iterrows():
        odd = row['ODD']
        cible = row['Cible']
        indicateur = row['Indicateur']
        mot_cle = row['Mots-cles']

         # Nous utilisons MERGE au lieu de CREATE afin de créer les nœuds ODD, Cible, Indicateur, Mots-clés sans doublons. Si le nœud existe déjà, il ne sera pas recréé
        graph.run(f"MERGE (o:SDG {{code: '{odd[:5]}'}}) SET o.title = '{odd[8:]}'")
        graph.run(f"MERGE (c:Target {{code: '{cible[6:12]}'}}) SET c.description = '{cible[14:]}'")
        graph.run(f"MERGE (i:Indicator {{code: '{indicateur[11:19]}'}}) SET i.description = '{indicateur[22:]}'")
        graph.run(f'MERGE (m:Keyword {{word: "{mot_cle}"}})')

        # Les relations entre les nœuds
        graph.run(f"""
            MATCH (o:SDG {{code: '{odd[:5]}'}})
            MATCH (c:Target {{code: '{cible[6:12]}'}})
            MATCH (i:Indicator {{code: '{indicateur[11:19]}'}})
            MATCH (m:Keyword {{word: "{mot_cle}"}})
            MERGE (o)-[:contain]->(c)
            MERGE (c)-[:contain]->(i)
            MERGE (i)-[:measuredBy]->(m)
        """)


In [5]:
Ontology_final = "../Other/Ontology_final.csv"

with open(Ontology_final, encoding="utf-8") as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter=';')
    data = list(csvreader)

df_Ontology_final = pd.DataFrame(data)

In [6]:
for _, row in df_Ontology_final.iterrows():
    mot_cle = row['﻿keyword']
    classification = row['classification']
    graph.run(f'MERGE (k:Keyword {{word: "{mot_cle}"}})')
    graph.run(f"MERGE (g:Goal {{title: '{classification}'}})")
    graph.run(f"""
    MATCH (k:Keyword {{word: "{mot_cle}"}}), (g:Goal {{title: "{classification}"}})
    MERGE (g)-[:contain]->(k)
    """)
